http://anie.me/On-Torchtext/

Can think of each character as a symbol, each word a line.

In [1]:
from torchtext import data

In [2]:
class Arguments():
    def __init__(self, batch_size):
        self.batch_size = batch_size
        
args = Arguments(32)

In [3]:
TEXT = data.Field(batch_first=True, lower=True, tokenize="spacy") #by default tokenize simply splits on spaces, this uses the spaCy tokenizer, but must be installed via `conda install spacy`
LABELS = data.Field(batch_first=True, pad_token=None, unk_token=None) #don't want the <pad> or <unk> tokens in the labels



    Only loading the 'en' tokenizer.



In [4]:
train, dev, test = data.TabularDataset.splits(
    path='data/processed/', train='train.txt',
    validation='dev.txt', test='test.txt', format='tsv',
    fields=[('sentence1', TEXT), ('sentence2', TEXT), ('labels', LABELS)])

In [5]:
print(train.fields)

{'sentence1': <torchtext.data.field.Field object at 0x7f43409c8748>, 'sentence2': <torchtext.data.field.Field object at 0x7f43409c8748>, 'labels': <torchtext.data.field.Field object at 0x7f433b084668>}


In [6]:
print(len(train))

549367


In [7]:
print(vars(train[0]))

{'sentence1': ['a', 'person', 'on', 'a', 'horse', 'jumps', 'over', 'a', 'broken', 'down', 'airplane', '.'], 'sentence2': ['a', 'person', 'is', 'training', 'his', 'horse', 'for', 'a', 'competition', '.'], 'labels': ['neutral']}


In [8]:
train_iter, dev_iter, test_iter = data.Iterator.splits((train, dev, test), #the dataset object to load data from
                                                            batch_size = args.batch_size, 
                                                            #sort_key=lambda x: len(x.sentence1), #how should it organise the buckets?
                                                            device=-1) #-1 implies using the CPU

In [9]:
TEXT.build_vocab(train.sentence1, vectors="glove.6B.100d")
TEXT.build_vocab(train.sentence2, vectors="glove.6B.100d")
LABELS.build_vocab(train.labels)

"""
To actually load the pre-trained vectors:

vocab = TEXT.vocab
self.embed = nn.Embedding(len(vocab), emb_dim)
self.embed.weight.data.copy_(vocab.vectors)
"""

'\nTo actually load the pre-trained vectors:\n\nvocab = TEXT.vocab\nself.embed = nn.Embedding(len(vocab), emb_dim)\nself.embed.weight.data.copy_(vocab.vectors)\n'

In [10]:
print(len(TEXT.vocab))

30335


In [11]:
TEXT.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index>,
            {'<unk>': 0,
             '<pad>': 1,
             'a': 2,
             '.': 3,
             'the': 4,
             'is': 5,
             'are': 6,
             'man': 7,
             'in': 8,
             'on': 9,
             'people': 10,
             'woman': 11,
             'of': 12,
             'to': 13,
             'two': 14,
             'at': 15,
             'and': 16,
             'with': 17,
             'his': 18,
             'playing': 19,
             'men': 20,
             'girl': 21,
             'for': 22,
             'boy': 23,
             'there': 24,
             'dog': 25,
             'an': 26,
             'outside': 27,
             'wearing': 28,
             'person': 29,
             'sitting': 30,
             'her': 31,
             'group': 32,
             'women': 33,
             'young': 34,
             'child': 35,
             'their': 36,
             'walking': 37,
    

In [14]:
print(LABELS.vocab.stoi)

defaultdict(<function _default_unk_index at 0x7f4340a03488>, {'entailment': 0, 'contradiction': 1, 'neutral': 2})


In [17]:
import torch
import torch.nn as nn

class Net(nn.Module):
    def __init__(self, input_size):
        super(Net, self).__init__()
        
        self.input_size = input_size

    def forward(self, x):
        return x
    
net = Net(100)

In [18]:
batch = next(iter(train_iter))
print(batch.sentence1)
print(batch.sentence2)
print(batch.labels)

Variable containing:

Columns 0 to 10 
    20     16     33     27     52    201     13   2240   2553      3      1
     2   5197    972     12     44     20      8     64    440     16   2665
     2      7      8    361     16      2     60    378     17      4   2283
     2     34     43    107      9      2   6140   1814      3      1      1
     4    239      5     38      9      2    636      9      2    177    172
    10     97     27      2     90    109     13      2   1233      3      1
     2     34      7      5    123      2    512    363     46     14    230
    14     10      6    474      2    693      8     73     12      4   3623
    14     42      6     38      9      4    855    109     13      2    614
    41     10     17     53   3059     47    656     53     16     64   3059
    26    493     97     13   4818      2     29      9      4     45      3
     2     34     23      5    296    125     12      2    820      8      2
     2      7    167      2    717   

In [20]:
type(batch.sentence1)

torch.autograd.variable.Variable